In [ ]:
import requests
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
from IPython.display import Image
import re
import time
import autoit
from selenium import webdriver
from selenium.webdriver.chrome.options import *
from selenium.webdriver.common.keys import Keys
import operator

import tweepy as tw
import bs4
import requests

consumer_key= 'your-consumer-key'
consumer_secret= 'your-consumer-secret'
access_token= 'your-access-token'
access_token_secret= 'your-access-token-secret'
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
key_word = input('Please enter your key word:')

In [ ]:
def clean_input(key_word):
    tag = key_word.replace(" ", "")
    if tag.startswith('#'):
        return tag[1:].lower()
    else:
        return tag.lower()

In [ ]:
def get_posts(key_word):
    url= 'https://www.instagram.com/explore/tags/'+key_word+'/'
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    script = soup.find('script', text=lambda t: \
                           t.startswith('window._sharedData'))
    page_json = script.text.split(' = ', 1)[1].rstrip(';')
    posts=json.loads(page_json)
    b=posts['entry_data']['TagPage'][0]['graphql']['hashtag']['edge_hashtag_to_top_posts']['edges']
    return b

In [ ]:
def get_top_post(key_word):
    posts= get_posts(key_word)
    l=[]
    for item in posts:
        d={}
        d['likes']=item['node']['edge_liked_by']['count']
        d['url']=item['node']['display_url']
        d['urlcode']=item['node']['shortcode']
        d['owner']=item['node']['owner']['id']
        d['caption']=item['node']['edge_media_to_caption']['edges'][0]['node']['text']
        l.append(d)
        l.sort(key=operator.itemgetter('likes'), reverse=True)
    return l[0]

In [ ]:
def download_image(url):
    f = open('download.jpg','wb')
    f.write(requests.get(url).content)
    f.close()

In [ ]:
def get_owner(shortcode):
    url= 'https://www.instagram.com/p/'+shortcode+'/'
    html = urllib.request.urlopen(url).read()

    soup = BeautifulSoup(html, 'html.parser')
    script = soup.find('script', text=lambda t: \
                           t.startswith('window._sharedData'))
    page_json = script.text.split(' = ', 1)[1].rstrip(';')

    post=json.loads(page_json)
    return post['entry_data']['PostPage'][0]['graphql']['shortcode_media']['owner']['username']

In [ ]:
def return_all_hashtags(tweets, tag):
    all_hashtags = []
    for tweet in tweets:
        for word in tweet.split():
            if word.startswith('#') and word.lower() != '#' + tag.lower():
                all_hashtags.append(word.lower())
    return all_hashtags

def extract_shared_data(doc):
    for script_tag in doc.find_all("script"):
        if script_tag.text.startswith("window._sharedData ="):
            shared_data = re.sub("^window\._sharedData = ", "", script_tag.text)
            shared_data = re.sub(";$", "", shared_data)
            shared_data = json.loads(shared_data)
            return shared_data

def get_hashtags(key_word):
    tweets = tw.Cursor(api.search,
                       q='#' + key_word,
                       lang="en").items(200)
    tweets_list = []
    for tweet in tweets:
        tweets_list.append(tweet.text)

    url_string = "https://www.instagram.com/explore/tags/%s/" % key_word
    response = bs4.BeautifulSoup(requests.get(url_string).text, "html.parser")

    shared_data = extract_shared_data(response)
    media = shared_data['entry_data']['TagPage'][0]['graphql']['hashtag']['edge_hashtag_to_media']['edges']

    captions = []
    for post in media:
        if post['node']['edge_media_to_caption']['edges'] != []:
            captions.append(post['node']['edge_media_to_caption']['edges'][0]['node']['text'])

    all_tags = return_all_hashtags(tweets_list + captions, key_word)
    frequency = {}
    for item in set(all_tags):
        frequency[item] = all_tags.count(item)
    return {k: v for k, v in sorted(frequency.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
def make_post(key_word):
    tag = clean_input(key_word)
    top_post = get_top_post(tag)
    download_image(top_post['url'])
    caption = 'Repost from @'+get_owner(top_post['urlcode']) + ' \n' + ', '.join(list(get_hashtags(tag).keys())[:10])
    return caption

In [ ]:
class Autogram:

    def __init__(self, username, password, dry_run=False):
        self.username = username
        self.password = password
        self.dry_run = dry_run  # WARNING: Set to False if you want to upload for real!
        self.driver = webdriver.Chrome('chromedriver.exe', options=self._get_options())
        self.default_hashtags = [
            '#python', '#programming', '#pythonprogramming', '#developer', '#softwareengineer', '#dev', '#python3' 
            '#pythonsofinstagram', '#pip', '#coding', '#coder', '#coderlifestyle', '#coderslife', '#softwaredeveloper'
        ]

    def set_username(self, usr):
        self.username = usr

    def set_password(self, pwd):
        self.password = pwd

    @staticmethod
    def _get_options():
        options = Options()
        options.add_argument("--log-level=3")
        options.add_argument("--silent")
        #options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-logging")
        options.add_argument("--mute-audio")
        #mobile_emulation = {"deviceName": "Nexus 5"}
        #options.add_experimental_option("mobileEmulation", mobile_emulation)
        options.add_argument('--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1')

        return options

    def open_instagram(self):
        print('Opening Instagram...')
        self.driver.get("https://www.instagram.com/accounts/login/?hl=en")
        time.sleep(3)

    def login(self):
        print('Logging into Instagram...')
        a=self.driver.find_elements_by_css_selector('form input')
        emailInput = a[0]
        passwordInput = a[1]
        emailInput.send_keys(self.username)
        passwordInput.send_keys(self.password)
        passwordInput.send_keys(Keys.ENTER)
        
    def profile_page(self):
        self.driver.find_element_by_xpath('//*[@id="react-root"]/section/nav[2]/div/div/div[2]/div/div/div[5]').click()
        time.sleep(0.5)

    def popup_close_save_login_info(self):
        print('Trying to close `save login info` popup...')
        attempts = 0
        while True:
            try:
                time.sleep(0.5)
                self.driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/section/div/button').click()
                break
            except:
                pass
                attempts += 1
                if attempts > 5:
                    print('Did not find `save login info` popup')
                    break

    def popup_close_turn_on_notifications(self):
        print('Trying to close `turn on notifications` popup...')
        attempts = 0
        while True:
            try:
                time.sleep(0.5)
                self.driver.find_element_by_xpath('/html/body/div[4]/div/div/div[3]/button[2]').click()
                break
            except:
                pass
                attempts += 1
                if attempts > 5:
                    print('Did not find `turn on notifications` popup')
                    break

    def popup_close_add_to_home_screen(self):
        print('Trying to close `add to home screen` popup...')
        attempts = 0
        while True:
            try:
                time.sleep(0.5)
                self.driver.find_element_by_xpath('/html/body/div[4]/div/div/div[3]/button[2]').click()
                break
            except:
                pass
                attempts += 1
                if attempts > 5:
                    print('Did not find `add to home screen` popup')
                    break

    def upload_image(self, file_path, description=''):
        # 1. Click upload image button
        print('Clicking `upload` button...')
        self.driver.find_element_by_xpath("//div[@role='menuitem']").click()
        time.sleep(2)

        # 2. Get the windows file explorer window that opened
        # Add the image path and click enter
        print('Uploading image to file explorer...')
        autoit.win_active("Open")
        autoit.control_send("Open", "Edit1", file_path)
        autoit.control_send("Open", "Edit1", "{ENTER}")
        time.sleep(2)

        # 3. Image should import and click Next button
        print('Image importing and processed to description...')
        self.driver.find_element_by_xpath('//*[@id="react-root"]/section/div[1]/header/div/div[2]/button').click()
        time.sleep(2)

        # 4. Add description
        print('Adding description...')
        self.driver.find_element_by_xpath('//*[@id="react-root"]/section/div[2]/section[1]/div[1]/textarea').click()
        self.driver.find_element_by_xpath('//*[@id="react-root"]/section/div[2]/section[1]/div[1]/textarea').send_keys(description)
        time.sleep(2)

        # 5. CLick Share
        print('Clicking `Share`!')
        if self.dry_run:
            print('DRY RUN IS ENABLED! - Therefore, not actually posting to IG.')
        else:
            self.driver.find_element_by_xpath('//*[@id="react-root"]/section/div[1]/header/div/div[2]/button').click()
        time.sleep(4)
        print('Upload completed.')

    def _auto_post(self, image_path, description):
        self.open_instagram()
        self.login()
        self.popup_close_save_login_info()
        self.popup_close_turn_on_notifications()
        self.popup_close_add_to_home_screen()
        self.upload_image(image_path, description=description)
        self.popup_close_turn_on_notifications()
        self.profile_page()

In [ ]:
ig = Autogram('your-instagram-username', 'your-instagram-password')
ig.open_instagram()
ig.login()
ig.popup_close_save_login_info()
ig.popup_close_turn_on_notifications()
ig.popup_close_add_to_home_screen()

In [ ]:
ig.upload_image(os.path.normpath(r'full-path-to-the-downloaded-photo\download.jpg'), description=make_post(key_word))
ig.popup_close_turn_on_notifications()